## Initialisierung

In [21]:
import gzip
import numpy as np
import pandas as pd
from sqlalchemy import create_engine#, text, MetaData, Table, Column, String
from geopy.geocoders import Nominatim
import seaborn as sns
import matplotlib.pyplot as plt
import holidays
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [3]:
sql_password = '' # Benutze hier dein MySQL- Passwort
engine = create_engine('mysql+mysqlconnector://root:' + sql_password + '@localhost:3306/verkehrsprojekt')
connection = engine.connect()

## Machine Learning

#### Dataframe erstellen

In [4]:
# pkw- Spalte

query = f"""
SELECT  
    timestamp, Durchschnitt
FROM 
    pkw_daten
"""
df = pd.read_sql(query,engine)
df = df.rename(columns = {'Durchschnitt':'Anzahl PKW'})
df  = df.set_index('timestamp')
pkw_spalte = df

In [5]:
# Fahrrad- Spalte

query = f"""
SELECT  
    timestamp, Durchschnitt
FROM 
    fahrraddaten
"""
df = pd.read_sql(query,engine)
df = df.rename(columns = {'Durchschnitt':'Anzahl Fahrräder'})
df  = df.set_index('timestamp')
fahrrad_spalte = df

In [6]:
# Dataframe für Machine learning erstellen

df = pd.read_csv('Bezirke_Durchschnitt.csv', decimal = '.' )
df['time'] = pd.to_datetime(df['time'])
df['precipitation (mm)'] = df['rain (mm)'] + 10*df['snowfall (cm)']
df = df.drop(columns = ['rain (mm)', 'snowfall (cm)'])
df['dayofweek'] = df['time'].dt.dayofweek
df['month'] = df['time'].dt.month
df['hour'] = df['time'].dt.hour 
berlin_holidays = holidays.Germany(state = 'BE')
df['is_holiday'] = df['time'].apply(lambda x: berlin_holidays.get(x, None))
df['is_holiday'] = df['is_holiday'].apply(lambda x: 1 if isinstance(x, str) else 0) # Es ist komisch, dass ich das in zwei Schritten machen muss. Aber die isin- methode hat komische Ergebnisse produziert
#df['is_holiday'] = ((df['is_holiday'] == 1) | (df['dayofweek'] == 6)).astype(int) # Auch Sonntage sollen als Holiday gewertet werden
df = df.rename(columns = {'time':'timestamp'})
df = df.set_index('timestamp')
df = pd.concat([df,pkw_spalte,fahrrad_spalte], axis = 1)

In [7]:
# NaN filtern
df = df[~df['Anzahl PKW'].isna()]
df = df[~df['Anzahl Fahrräder'].isna()]

In [8]:
# Aureißer ausschließen
problem_tage = [
'2018-04-25',
'2019-07-28',
'2019-10-20',
'2021-12-13',
'2023-01-30',
'2023-05-15',
]
problem_tage = pd.to_datetime(problem_tage)
filt = df.index.normalize().isin(problem_tage)
df = df[~filt]

In [9]:
df_unscaled = df.copy()

#### Betrachte den Dataframe

In [110]:
df.head()

,temperature_2m (°C),relative_humidity_2m (%),cloud_cover (%),precipitation (mm),dayofweek,month,hour,is_holiday,Anzahl PKW,Anzahl Fahrräder
timestamp,,,,,,,,,,
2018-01-01 00:00:00,11.066667,71.666667,61.333333,0.000000,0,1,0,1,183.208,6.538462
2018-01-01 01:00:00,11.141667,70.000000,77.250000,0.016667,0,1,1,1,357.316,10.730769
2018-01-01 02:00:00,11.591667,64.500000,92.416667,0.000000,0,1,2,1,359.928,15.153846
2018-01-01 03:00:00,11.825000,62.083333,95.916667,0.000000,0,1,3,1,284.856,13.269231
2018-01-01 04:00:00,11.641667,63.416667,93.916667,0.000000,0,1,4,1,225.944,8.115385


In [55]:
#sns.pairplot(df, plot_kws={"s": 0.05})

In [111]:
df.describe()

,temperature_2m (°C),relative_humidity_2m (%),cloud_cover (%),precipitation (mm),dayofweek,month,hour,is_holiday,Anzahl PKW,Anzahl Fahrräder
count,51886.000000,51886.000000,51886.000000,51886.000000,51886.000000,51886.000000,51886.000000,51886.000000,51886.000000,51886.000000
mean,10.993283,73.602441,65.410857,0.090869,3.007170,6.528447,11.512219,0.027715,440.466693,90.083151
std,8.250195,18.149008,37.809799,0.407510,1.997962,3.454696,6.920103,0.164155,247.726517,79.933258
min,-14.375000,14.583333,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,20.661597,0.115385
25%,4.591667,60.833333,29.666667,0.000000,1.000000,4.000000,6.000000,0.000000,187.627605,19.213942
50%,10.375000,77.666667,84.333333,0.000000,3.000000,7.000000,12.000000,0.000000,475.621792,70.423077
75%,17.181250,89.166667,99.750000,0.000000,5.000000,10.000000,18.000000,0.000000,657.805121,141.133333
max,37.358333,100.000000,100.000000,13.691667,6.000000,12.000000,23.000000,1.000000,923.629482,423.500000


In [131]:
#sns.regplot(x = df['relative_humidity_2m (%)'], y = df['Anzahl Fahrräder'], order = 1, scatter_kws={"s": 0.01})

#### Normalisiere den Dataframe

In [112]:
scaler = StandardScaler()
df[['precipitation (mm)']] = scaler.fit_transform(df[['precipitation (mm)']])
df[['temperature_2m (°C)']] = scaler.fit_transform(df[['temperature_2m (°C)']])

In [113]:
scaler = MinMaxScaler()
df[['relative_humidity_2m (%)','cloud_cover (%)']] = scaler.fit_transform(df[['relative_humidity_2m (%)','cloud_cover (%)']])

In [36]:
#Stelle Anzahl PKW, Fahrräder anteilig an ihrere Gesamtsumme dar
df['Anzahl PKW'] , df['Anzahl Fahrräder'] = df['Anzahl PKW'] / ( df['Anzahl PKW'] + df['Anzahl Fahrräder'] ) , df['Anzahl Fahrräder'] / ( df['Anzahl PKW'] + df['Anzahl Fahrräder'] )

C:\Users\Efi\AppData\Local\Temp\ipykernel_2500\3043990704.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Anzahl PKW'] , df['Anzahl Fahrräder'] = df['Anzahl PKW'] / ( df['Anzahl PKW'] + df['Anzahl Fahrräder'] ) , df['Anzahl Fahrräder'] / ( df['Anzahl PKW'] + df['Anzahl Fahrräder'] )


In [62]:
#df['temperature_2m (°C)'].hist(bins = 200, density = False)

In [63]:
#df['precipitation (mm)'][df['precipitation (mm)'] >= 0].hist(bins = 200)

In [64]:
#df['snowfall (cm)'][df['snowfall (cm)'] >= 0].hist(bins = 200)

#### Automatisierte Analyse

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

In [27]:
s_scaler = StandardScaler()
m_scaler = MinMaxScaler()

In [28]:
#scaling_methods = ['keine', 'minmax', 'standard']
scaling_methods = ['keine']

In [29]:
wetter_features = df_unscaled[['temperature_2m (°C)', 'relative_humidity_2m (%)', 'cloud_cover (%)', 'precipitation (mm)']]
zeit_features = df_unscaled[['dayofweek', 'month', 'hour']]
zeit_features = pd.concat( [ zeit_features.drop('dayofweek', axis = 1), pd.get_dummies(zeit_features['dayofweek'], prefix = 'dow', dtype = int) ], axis = 1)
zeit_features = pd.concat( [ zeit_features.drop('month', axis = 1), pd.get_dummies(zeit_features['month'], prefix = 'month', dtype = int) ], axis = 1)
zeit_features = pd.concat( [ zeit_features.drop('hour', axis = 1), pd.get_dummies(zeit_features['hour'], prefix = 'hour', dtype = int) ], axis = 1)

In [30]:
# Untersuchtes Fahrzeug: PKW oder Fahrrad? Aktiviere/Deaktiviere die jeweileige Zeile
fahrzeug = 'PKW'
#fahrzeug = 'Fahrräder'

column_names = ['temperature_2m (°C)', 'relative_humidity_2m (%)', 'cloud_cover (%)', 'precipitation (mm)', 'Anzahl ' + fahrzeug]

In [31]:
# Welche Features sollen berücksichtigt werden?
use_weather_features = True
use_time_features = True

In [ ]:
# Hier kennst du dich auf die entscheidenden konzentrieren, um die runtime zu erhöhen

In [251]:
####################################################################################################################### Original
best_r2 = - float("inf")
worst_r2 = float("inf")
method_collection = {name: None for name in column_names}
for method_temp in scaling_methods:
    for method_hum in scaling_methods:
        for method_cloud in scaling_methods:
            for method_prec in scaling_methods:
                for method_num in scaling_methods:

                    # Erstelle das df, je nachdem, welche Features du benutzt
                    df = df_unscaled.drop(columns = df_unscaled.columns)
                    if use_weather_features:
                        df[wetter_features.columns] = wetter_features
                    if use_time_features:
                        df[zeit_features.columns] = zeit_features
                    df[[f'Anzahl {fahrzeug}']] = df_unscaled[[f'Anzahl {fahrzeug}']]
                    
                    method_collection['temperature_2m (°C)'] = method_temp
                    method_collection['relative_humidity_2m (%)'] = method_hum
                    method_collection['cloud_cover (%)'] = method_cloud
                    method_collection['precipitation (mm)'] = method_prec
                    method_collection[f'Anzahl {fahrzeug}'] = method_num

                    if use_weather_features:
                        for name in column_names:
                            if method_collection[name] == 'keine':
                                pass
                            elif method_collection[name] == 'minmax':
                                df[[name]] = m_scaler.fit_transform(df[[name]])
                            elif method_collection[name] == 'standard':
                                df[[name]] = s_scaler.fit_transform(df[[name]])

                    X = df.drop(columns = [f'Anzahl {fahrzeug}'])
                    y = df[[f'Anzahl {fahrzeug}']]
                    
                    split_filt = ( y.index < '2023-01-01 00:00:00' )
                    X_train, y_train = X[split_filt], y[split_filt]
                    X_test, y_test = X[~split_filt], y[~split_filt]

                    model = SVR(kernel='rbf', C=1.0, epsilon=0.1)
                    model.fit(X_train, y_train)
                    y_pred = model.predict(X_test)
                    mse = mean_squared_error(y_test, y_pred)
                    sd = np.sqrt(mse)
                    r2 = r2_score(y_test, y_pred)

                    worst_r2 = min(r2,worst_r2)
                    if r2 > best_r2:
                        best_method_collection = method_collection
                        best_r2 = r2


Verwendete_Features = ['Wetter' if use_weather_features else ''] + ['Zeit' if use_time_features else '']
print(f"Verwendeter Algorithmus: SVR")
print(f"Verwendete Features: {Verwendete_Features}")
print(f"Betrachtetes Fahrzeug: {fahrzeug}")
if use_weather_features:
    print([best_method_collection[key]for key in best_method_collection])
print(f"bester R^2: {best_r2}")
print(f"schlechtester R^2: {worst_r2}")
print("")

C:\Users\Efi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Verwendeter Algorithmus: SVR
Verwendete Features: ['Wetter', 'Zeit']
Betrachtetes Fahrzeug: PKW
['keine', 'keine', 'keine', 'keine', 'keine']
bester R^2: 0.16531284833973203
schlechtester R^2: 0.16531284833973203



In [35]:
print(best_parameter_collection)
print(best_r2)

['linear', 0.1, 0.001, 0.001]
0.6785160748454775


ähnlich gut mit C = 100, kernel linear

In [ ]:
### speziell SVR
scaling_methods = ['keine','minmax','standard']
kernel_types = ['rbf']#['sigmoid', 'poly','linear', 'rbf']
C_values = [0.1, 1, 10, 100]
epsilon_values = [0.001, 0.01, 0.1, 0.5, 1]
gamma_values = [0.001, 0.01, 0.1, 1]

best_r2 = - float("inf")
worst_r2 = float("inf")
method_collection = {name: None for name in column_names}

for kernel in kernel_types:
    for C in C_values:
        for epsilon in epsilon_values:
            for gamma in gamma_values:
                for method_temp in ['keine']:
                    for method_hum in ['minmax']:
                        for method_cloud in ['minmax']:
                            for method_prec in ['keine']:
                                for method_num in ['keine']:
                
                                    # Erstelle das df, je nachdem, welche Features du benutzt
                                    df = df_unscaled.drop(columns = df_unscaled.columns)
                                    if use_weather_features:
                                        df[wetter_features.columns] = wetter_features
                                    if use_time_features:
                                        df[zeit_features.columns] = zeit_features
                                    df[[f'Anzahl {fahrzeug}']] = df_unscaled[[f'Anzahl {fahrzeug}']]
                                    
                                    method_collection['temperature_2m (°C)'] = method_temp
                                    method_collection['relative_humidity_2m (%)'] = method_hum
                                    method_collection['cloud_cover (%)'] = method_cloud
                                    method_collection['precipitation (mm)'] = method_prec
                                    method_collection[f'Anzahl {fahrzeug}'] = method_num
                
                                    if use_weather_features:
                                        for name in column_names:
                                            if method_collection[name] == 'keine':
                                                pass
                                            elif method_collection[name] == 'minmax':
                                                df[[name]] = m_scaler.fit_transform(df[[name]])
                                            elif method_collection[name] == 'standard':
                                                df[[name]] = s_scaler.fit_transform(df[[name]])
                
                                    X = df.drop(columns = [f'Anzahl {fahrzeug}'])
                                    y = df[[f'Anzahl {fahrzeug}']]
                                    
                                    split_filt = ( y.index < '2023-01-01 00:00:00' )
                                    X_train, y_train = X[split_filt], y[split_filt]
                                    X_test, y_test = X[~split_filt], y[~split_filt]
                
                                    model = SVR(kernel=kernel, C=C, epsilon=epsilon)
                                    model.fit(X_train, y_train)
                                    y_pred = model.predict(X_test)
                                    mse = mean_squared_error(y_test, y_pred)
                                    sd = np.sqrt(mse)
                                    r2 = r2_score(y_test, y_pred)
                
                                    worst_r2 = min(r2,worst_r2)
                                    if r2 > best_r2:
                                        best_parameter_collection = [kernel,C,epsilon,gamma]
                                        best_r2 = r2
                                    print(f"tested: kernel = {kernel}, C = {C}, epsilon = {epsilon}, gamma = {gamma}")
                                    print(f"R^2 = {r2}")
                                    print("")
                                    print("")


Verwendete_Features = ['Wetter' if use_weather_features else ''] + ['Zeit' if use_time_features else '']
print(f"Verwendeter Algorithmus: SVR")
print(f"Verwendete Features: {Verwendete_Features}")
print(f"Betrachtetes Fahrzeug: {fahrzeug}")
if use_weather_features:
    print([best_method_collection[key]for key in best_method_collection])
print(f"bester R^2: {best_r2}")
print(f"schlechtester R^2: {worst_r2}")
print("")

C:\Users\Efi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


tested: kernel = rbf, C = 0.1, epsilon = 0.001, gamma = 0.001
R^2 = 0.0052600135098281875




C:\Users\Efi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


tested: kernel = rbf, C = 0.1, epsilon = 0.001, gamma = 0.01
R^2 = 0.0052600135098281875




C:\Users\Efi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#### Verschiedene Algorithmen

In [38]:
df = pd.concat( [ df.drop('dayofweek', axis = 1), pd.get_dummies(df['dayofweek'], prefix = 'dow', dtype = int) ], axis = 1)
df = pd.concat( [ df.drop('month', axis = 1), pd.get_dummies(df['month'], prefix = 'month', dtype = int) ], axis = 1)
df = pd.concat( [ df.drop('hour', axis = 1), pd.get_dummies(df['hour'], prefix = 'hour', dtype = int) ], axis = 1)

In [60]:
#X = df[['temperature_2m (°C)','relative_humidity_2m (%)','rain (mm)','snowfall (cm)','cloud_cover (%)']]
#X = df.drop(columns = ['temperature_2m (°C)','relative_humidity_2m (%)','rain (mm)','snowfall (cm)','cloud_cover (%)', 'Anzahl PKW', 'Anzahl Fahrräder'])
X = df.drop(columns = ['Anzahl PKW', 'Anzahl Fahrräder'])
y = df[['Anzahl PKW']]
#y = df[['Anzahl Fahrräder']]
#y = df[['Anzahl PKW','Anzahl Fahrräder']]

split_filt = ( y.index < '2023-01-01 00:00:00' )
X_train, y_train = X[split_filt], y[split_filt]
X_test, y_test = X[~split_filt], y[~split_filt]

##### Lineare Regression

In [44]:
if False:
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    sd = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    # Ergebnisse
    print(f"Standard Deviation: {sd}")
    print("")
    print(f"R^2 Score: {r2}")
    print("")
    print("")
    #print("")
    #print("Koeffizienten:", model.coef_)
    #print("Achsenabschnitt (Intercept):", model.intercept_)

Standard Deviation: 0.04785495221567448

R^2 Score: 0.6808519360456639


##### KNN

In [ ]:
# 11 ?

In [46]:
if False:
    knn = KNeighborsRegressor(n_neighbors=20)  # n_neighbors ist die Anzahl der Nachbarn

    # Modell trainieren
    knn.fit(X_train, y_train)
    
    # Vorhersagen treffen
    y_pred = knn.predict(X_test)
    
    # Performance bewerten
    mse = mean_squared_error(y_test, y_pred)
    sd = np.sqrt(mse)
    print(f"Standardabweichung: {sd}")

Standardabweichung: 0.04031627131767241


In [ ]:
if False:
    param_grid = {'n_neighbors': range(1, 21)}
    grid_search = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    print(f"Best Parameters: {grid_search.best_params_}")

##### Random Forest

In [ ]:
# Beste Parameter: 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200

In [215]:
from time import time
if False:
    begin = time()
    rf = RandomForestRegressor(n_estimators=200, random_state=42, max_depth = 10, min_samples_leaf = 1, min_samples_split = 10)
    
    # Modell trainieren
    rf.fit(X_train, y_train)
    
    # Vorhersagen treffen
    y_pred = rf.predict(X_test)
    
    # Performance evaluieren
    mse = mean_squared_error(y_test, y_pred)
    sd = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    end = time()
    print(f"Standardabweichung: {sd}")
    print(f"R^2 Score: {r2}")

C:\Users\Efi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Standardabweichung: 51.58040659092073
R^2 Score: 0.6245722261523179


In [ ]:
if False:
    param_grid = {
        'n_estimators': [50, 100, 200],  # Anzahl der Bäume
        'max_depth': [None, 10, 20],    # Maximale Tiefe
        'min_samples_split': [2, 5, 10],  # Mindestanzahl von Stichproben für Split
        'min_samples_leaf': [1, 2, 4]    # Mindestanzahl von Stichproben in einem Blatt
    }
    
    grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    
    print(f"Beste Parameter: {grid_search.best_params_}")

##### SVR

In [220]:
if False:
    begin = time()
    # SVR-Modell initialisieren (mit RBF-Kernel)
    svr = SVR(kernel='rbf', C=1.0, epsilon=0.1)
    
    # Modell trainieren
    svr.fit(X_train, y_train)
    
    # Vorhersagen treffen
    y_pred = svr.predict(X_test)
    
    # Performance evaluieren
    mse = mean_squared_error(y_test, y_pred)
    sd = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    end = time()
    print(end-begin)
    print(f"Standard Deviation: {sd}")
    print(f"R^2 Score: {r2}")

C:\Users\Efi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


230.98116445541382
Standard Deviation: 42.39785361252609
R^2 Score: 0.746344304730622


##### Gradient Boost

In [221]:
if False:
    begin = time()
    gbr = GradientBoostingRegressor(
        n_estimators=100,     # Anzahl der Bäume
        learning_rate=0.1,    # Lernrate (Schrittgröße)
        max_depth=3,          # Maximale Tiefe der Bäume
        random_state=42
    )
    
    # Modell trainieren
    gbr.fit(X_train, y_train)
    
    # Vorhersagen treffen
    y_pred = gbr.predict(X_test)
    
    # Performance evaluieren
    mse = mean_squared_error(y_test, y_pred)
    sd = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    end = time()
    print(end-begin)
    print(f"Standardabweichung: {sd}")
    print(f"R^2 Score: {r2}")

C:\Users\Efi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


3.39844012260437
Standardabweichung: 45.505475372749416
R^2 Score: 0.7077973252935614


##### Anderes Modell

In [257]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.genmod.families.links import logit
from statsmodels.genmod.families import Binomial

model = smf.glm('y ~ X', data=data, family=Binomial(link=logit()))
results = model.fit()

# Ergebnisse anzeigen
print(results.summary())

C:\Users\Efi\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\genmod\families\links.py:13: FutureWarning: The logit link alias is deprecated. Use Logit instead. The logit link alias will be removed after the 0.15.0 release.
  warnings.warn(


PatsyError: Number of rows mismatch between data argument and y (740 versus 51886)
    y ~ X
    ^